In [ ]:
import pandas as pd
from feature_selectors import *
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
gebruikers_df = pd.read_csv(r'..\..\data\gebruikersinformatie.csv', delimiter = ';')
evenement_df = pd.read_csv(r'..\..\data\evenementinformatie.csv', delimiter = ';')
aanwezigheid_df = pd.read_csv(r'..\..\data\evenementaanwezigheid.csv', delimiter = ';')
bericht_df = pd.read_csv(r'..\..\data\berichtinteracties_met_sentiment.csv', delimiter = ';')

In [ ]:
aanwezigheid_df = merge_on_aanwezigheid(aanwezigheid_df, evenement_df, gebruikers_df)

aanwezigheid_df

In [ ]:
# Remove nonvalid entries
aanwezigheid_df = aanwezigheid_df[~(aanwezigheid_df['Gebruiker_Leeftijd'].isna())]
aanwezigheid_df = aanwezigheid_df[~(aanwezigheid_df['Gebruiker_Lidmaatschapstype'] == 'x')]
aanwezigheid_df

In [ ]:
# Convert ID to ints
to_convert = ["EvenementID", "GebruikerID", "Evenement_OrganisatorID"]

for col in to_convert:
    aanwezigheid_df[col] = aanwezigheid_df[col].str[1:].astype(int)
    
aanwezigheid_df

In [ ]:
# Convert columns to datetime datatype and extract month
date_columns = ['Evenement_EvenementDatum', 'Gebruiker_RegistratieDatum', 'Gebruiker_LaatsteLogin']

for col in date_columns:
    aanwezigheid_df[col] = pd.to_datetime(aanwezigheid_df[col], dayfirst = True)

for col in date_columns:
    aanwezigheid_df[f'{col}_month'] = aanwezigheid_df[col].dt.month

aanwezigheid_df

In [ ]:
# Create boolean dataframe
dummies_df = pd.DataFrame()

# Aanwezigheid columns
dummies_df = pd.concat([dummies_df, pd.get_dummies(aanwezigheid_df['Aanwezigheidsstatus'], prefix = 'Aanwezigheid')], axis = 1)

# Evenement columns
dummies_df = pd.concat([dummies_df, pd.get_dummies(aanwezigheid_df['Evenement_EvenementType'], prefix = 'EvenementType')], axis = 1)

dummies_df["EvenementDatum_is_weekday"] = aanwezigheid_df["Evenement_EvenementDatum"].dt.weekday < 5
dummies_df["EvenementDatum_is_weekend"] = aanwezigheid_df["Evenement_EvenementDatum"].dt.weekday >= 5

dummies_df = pd.concat([dummies_df, pd.get_dummies(aanwezigheid_df['Evenement_EvenementDatum_month'], prefix = 'EvenementDatum_month')], axis = 1)

# Gebruiker columns
dummies_df[f'Younger'] = aanwezigheid_df["Gebruiker_Leeftijd"] <= aanwezigheid_df["Gebruiker_Leeftijd"].mode()[0]
dummies_df[f'Older'] = aanwezigheid_df["Gebruiker_Leeftijd"] > aanwezigheid_df["Gebruiker_Leeftijd"].mode()[0]

dummies_df = pd.concat([dummies_df, pd.get_dummies(aanwezigheid_df['Gebruiker_Studierichting'].replace('Data engineering', 'Data Engineering'), prefix = 'Studierichting')], axis = 1)
dummies_df = pd.concat([dummies_df, pd.get_dummies(aanwezigheid_df['Gebruiker_Studiejaar'], prefix = 'Studiejaar')], axis = 1)
dummies_df = pd.concat([dummies_df, pd.get_dummies(aanwezigheid_df['Gebruiker_Lidmaatschapstype'], prefix = 'Lidmaatschapstype')], axis = 1)

dummies_df

In [ ]:
# Create frequent itemsets and association rules
freq_itemsets = apriori(dummies_df, min_support = 0.1, use_colnames = True)
association_rules(freq_itemsets, metric = 'lift', min_threshold = 1.25)